## Questão 02

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import random
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
## Função de treinamento
def train(model, criterion, optimizer, train_loader, val_loader, epochs=100):
    train_losses = []
    val_losses = []
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels.unsqueeze(1))
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        train_loss = running_loss / len(train_loader)
        train_losses.append(train_loss)

        val_loss = evaluate(model, criterion, val_loader)
        val_losses.append(val_loss)
        
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

    return train_losses, val_losses


## Função de avaliação
def evaluate(model, criterion, val_loader):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels.unsqueeze(1))
            running_loss += loss.item()
    return running_loss / len(val_loader)

### a)

Abaixo, a construção do dataset de pontos 2d rotulados de acordo com a função XOR.

In [3]:
n = 10000
data_X = []
data_y = []

for i in range(n):
    instance = np.array([random.uniform(-5,5), random.uniform(-5,5)])

    if instance[0]*instance[1] > 0:
        label = 1
    elif instance[0]*instance[1] < 0:
        label = 0
    else:
        continue

    data_X.append(instance)
    data_y.append(label)


data_X = np.array(data_X)
data_y = torch.tensor(data_y, dtype=torch.float32)

O código abaixo realiza as seguintes tarefas:

- Divide os dados em conjuntos de treino, validação e teste usando `train_test_split`.
- Converte os dados para tensores do PyTorch.
- Define a arquitetura de uma rede neural simples com uma camada oculta usando a classe `PerceptronXOR`.
- Instancia o modelo, a função de perda (no caso, Binary Cross-Entropy Loss) e o otimizador (Adam).
- Define o `batch_size` e cria os DataLoader para os conjuntos de treino e validação.
- Treina o modelo usando a função `train`, que itera sobre os dados de treino e realiza o backpropagation.
- Avalia o modelo nos dados de teste usando a função `evaluate`.
- Calcula a acurácia do modelo nos dados de teste

In [4]:
x_train, x_test, y_train, y_test = train_test_split(data_X, data_y, test_size=0.2, random_state=10)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=10)

x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
x_val_tensor = torch.tensor(x_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

class PerceptronXOR(nn.Module):
    def __init__(self):
        super(PerceptronXOR, self).__init__()
        self.fc1 = nn.Linear(2, 4)  ## Camada de entrada
        self.fc2 = nn.Linear(4, 1)  ## Camada de saída

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

## Instanciar o modelo, função de perda e otimizador
model = PerceptronXOR()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

batch_size = 32
train_loader = DataLoader(TensorDataset(x_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TensorDataset(x_val_tensor, y_val_tensor), batch_size=batch_size)

train_losses, val_losses = train(model, criterion, optimizer, train_loader, val_loader, epochs=300)

test_loader = DataLoader(TensorDataset(x_test_tensor, y_test_tensor), batch_size=batch_size)
test_loss = evaluate(model, criterion, test_loader)
print(f"\nTest Loss: {test_loss:.4f}")

model.eval()
with torch.no_grad():
    outputs = model(x_test_tensor)
    predicted = torch.round(outputs)
    accuracy = (predicted == y_test_tensor.unsqueeze(1)).sum().item() / predicted.size(0)
    print(f"Acurácia: {accuracy:.2f}")


C:\Users\jahon\AppData\Local\Temp\ipykernel_10092\901355331.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
C:\Users\jahon\AppData\Local\Temp\ipykernel_10092\901355331.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_val_tensor = torch.tensor(y_val, dtype=torch.float32)
C:\Users\jahon\AppData\Local\Temp\ipykernel_10092\901355331.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


Epoch 1/300, Train Loss: 0.7214, Val Loss: 0.6669
Epoch 2/300, Train Loss: 0.6261, Val Loss: 0.5905
Epoch 3/300, Train Loss: 0.5552, Val Loss: 0.5173
Epoch 4/300, Train Loss: 0.4863, Val Loss: 0.4513
Epoch 5/300, Train Loss: 0.4283, Val Loss: 0.3989
Epoch 6/300, Train Loss: 0.3829, Val Loss: 0.3572
Epoch 7/300, Train Loss: 0.3461, Val Loss: 0.3234
Epoch 8/300, Train Loss: 0.3160, Val Loss: 0.2955
Epoch 9/300, Train Loss: 0.2910, Val Loss: 0.2716
Epoch 10/300, Train Loss: 0.2697, Val Loss: 0.2523
Epoch 11/300, Train Loss: 0.2519, Val Loss: 0.2350
Epoch 12/300, Train Loss: 0.2363, Val Loss: 0.2205
Epoch 13/300, Train Loss: 0.2225, Val Loss: 0.2074
Epoch 14/300, Train Loss: 0.2101, Val Loss: 0.1957
Epoch 15/300, Train Loss: 0.1994, Val Loss: 0.1855
Epoch 16/300, Train Loss: 0.1902, Val Loss: 0.1766
Epoch 17/300, Train Loss: 0.1814, Val Loss: 0.1682
Epoch 18/300, Train Loss: 0.1734, Val Loss: 0.1608
Epoch 19/300, Train Loss: 0.1662, Val Loss: 0.1548
Epoch 20/300, Train Loss: 0.1599, Val Lo

In [5]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_test[y_test==0,0], y=x_test[y_test==0,1], mode='markers', name='0'))
fig.add_trace(go.Scatter(x=x_test[y_test==1,0], y=x_test[y_test==1,1], mode='markers', name='1'))
fig.update_layout(title='Dados de teste', xaxis_title='X', yaxis_title='Y', height=600, width=900)
fig.show()

In [7]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(train_losses)), y=train_losses, mode='lines', name='Train Loss'))
fig.add_trace(go.Scatter(x=np.arange(len(val_losses)), y=val_losses, mode='lines', name='Val Loss'))
fig.update_layout(title='Train and Val Loss', xaxis_title='Epoch', yaxis_title='Loss', height=600, width=1200)
fig.add_trace(go.Scatter(x=[len(train_losses)], y=[test_loss], mode='markers', marker=dict(color='orange', size=10), name='Test Loss'))
fig.show()

### b)

In [9]:
n=10000

data_X = []
data_y = []

for i in range(n):
    instance = np.array([random.uniform(1,10), random.uniform(0,1)])
    if np.log10(instance[0]) > instance[1]:
        label = 0
    else:
        label = 1

    data_X.append(instance)
    data_y.append(label)

x = np.array(data_X)
y = np.array(data_y)

In [10]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=10)

x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
x_val_tensor = torch.tensor(x_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Definir o modelo da rede neural
class PerceptronLog(nn.Module):
    def __init__(self):
        super(PerceptronLog, self).__init__()
        self.fc1 = nn.Linear(2, 4)  # Camada de entrada
        self.fc2 = nn.Linear(4, 1)  # Camada de saída

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

# Instanciar o modelo, função de perda e otimizador
model = PerceptronLog()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Definir o tamanho do batch e criar os DataLoader
batch_size = 32
train_loader = DataLoader(TensorDataset(x_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TensorDataset(x_val_tensor, y_val_tensor), batch_size=batch_size)

# Treinar o modelo
train_losses, val_losses = train(model, criterion, optimizer, train_loader, val_loader, epochs=300)

# Avaliar o modelo nos dados de teste
test_loader = DataLoader(TensorDataset(x_test_tensor, y_test_tensor), batch_size=batch_size)
test_loss = evaluate(model, criterion, test_loader)
print(f"\nTest Loss: {test_loss:.4f}")

model.eval()
with torch.no_grad():
    outputs = model(x_test_tensor)
    predicted = torch.round(outputs)
    accuracy = (predicted == y_test_tensor.unsqueeze(1)).sum().item() / predicted.size(0)
    print(f"Acurácia: {accuracy:.2f}")

Epoch 1/300, Train Loss: 0.5484, Val Loss: 0.5257
Epoch 2/300, Train Loss: 0.5250, Val Loss: 0.5051
Epoch 3/300, Train Loss: 0.5067, Val Loss: 0.4857
Epoch 4/300, Train Loss: 0.4881, Val Loss: 0.4669
Epoch 5/300, Train Loss: 0.4697, Val Loss: 0.4489
Epoch 6/300, Train Loss: 0.4519, Val Loss: 0.4304
Epoch 7/300, Train Loss: 0.4344, Val Loss: 0.4132
Epoch 8/300, Train Loss: 0.4184, Val Loss: 0.3969
Epoch 9/300, Train Loss: 0.4031, Val Loss: 0.3817
Epoch 10/300, Train Loss: 0.3888, Val Loss: 0.3675
Epoch 11/300, Train Loss: 0.3735, Val Loss: 0.3531
Epoch 12/300, Train Loss: 0.3594, Val Loss: 0.3393
Epoch 13/300, Train Loss: 0.3453, Val Loss: 0.3257
Epoch 14/300, Train Loss: 0.3314, Val Loss: 0.3121
Epoch 15/300, Train Loss: 0.3169, Val Loss: 0.2987
Epoch 16/300, Train Loss: 0.3033, Val Loss: 0.2864
Epoch 17/300, Train Loss: 0.2895, Val Loss: 0.2727
Epoch 18/300, Train Loss: 0.2771, Val Loss: 0.2607
Epoch 19/300, Train Loss: 0.2639, Val Loss: 0.2494
Epoch 20/300, Train Loss: 0.2515, Val Lo

In [11]:
## plot dos pontos de teste e a função log10(x) no intervalo [1,10]

fig = go.Figure()
fig.add_trace(go.Scatter(x=x_test[:,0], y=x_test[:,1], mode='markers', marker=dict(color=y_test, colorscale='Viridis', size=5), name='Dados de teste'))
x = np.linspace(1, 10, 100)
y = np.log10(x)
fig.add_trace(go.Scatter(x=x, y=y, mode='lines', line=dict(color='orange', width=3), name='log10(x)'))
fig.update_layout(title='Dados de teste e log10(x)', xaxis_title='X', yaxis_title='Y', height=600, width=900)
fig.show()

In [12]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(train_losses)), y=train_losses, mode='lines', name='Train Loss'))
fig.add_trace(go.Scatter(x=np.arange(len(val_losses)), y=val_losses, mode='lines', name='Val Loss'))
fig.update_layout(title='Train and Val Loss', xaxis_title='Epoch', yaxis_title='Loss', height=600, width=1200)
fig.add_trace(go.Scatter(x=[len(train_losses)], y=[test_loss], mode='markers', marker=dict(color='orange', size=10), name='Test Loss'))
fig.show()

### c)

In [13]:
n=10000

data_X = []
data_y = []

def f(x):
    return 10*x**5 + 5*x**4 + 2*x**3 - 0.5*x**2 + 3*x + 2

for i in range(n):
    instance = np.array([random.uniform(0,5), random.uniform(-5000,5000)])
    if f(instance[0]) > instance[1]:
        label = 0
    else:
        label = 1

    data_X.append(instance)
    data_y.append(label)

x = np.array(data_X)
y = torch.tensor(data_y, dtype=torch.float32)

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=10)

x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
x_val_tensor = torch.tensor(x_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Definir o modelo da rede neural
class PerceptronPol(nn.Module):
    def __init__(self):
        super(PerceptronPol, self).__init__()
        self.fc1 = nn.Linear(2, 128)  # Camada de entrada
        self.fc2 = nn.Linear(128, 128)  # Camada oculta
        self.fc3 = nn.Linear(128, 1)  # Camada de saída

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

# Instanciar o modelo, função de perda e otimizador
model = PerceptronPol()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Definir o tamanho do batch e criar os DataLoader
batch_size = 64
train_loader = DataLoader(TensorDataset(x_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TensorDataset(x_val_tensor, y_val_tensor), batch_size=batch_size)

# Treinar o modelo
train_losses, val_losses = train(model, criterion, optimizer, train_loader, val_loader, epochs=300)

# Avaliar o modelo nos dados de teste
test_loader = DataLoader(TensorDataset(x_test_tensor, y_test_tensor), batch_size=batch_size)
test_loss = evaluate(model, criterion, test_loader)
print(f"\nTest Loss: {test_loss:.4f}")

model.eval()
with torch.no_grad():
    outputs = model(x_test_tensor)
    predicted = torch.round(outputs)
    accuracy = (predicted == y_test_tensor.unsqueeze(1)).sum().item() / predicted.size(0)
    print(f"Acurácia: {accuracy:.2f}")

C:\Users\jahon\AppData\Local\Temp\ipykernel_10092\3383062971.py:5: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

C:\Users\jahon\AppData\Local\Temp\ipykernel_10092\3383062971.py:7: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

C:\Users\jahon\AppData\Local\Temp\ipykernel_10092\3383062971.py:9: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 1/300, Train Loss: 21.3355, Val Loss: 24.1598
Epoch 2/300, Train Loss: 21.6997, Val Loss: 24.2014
Epoch 3/300, Train Loss: 24.3233, Val Loss: 29.3793
Epoch 4/300, Train Loss: 28.7201, Val Loss: 29.3777
Epoch 5/300, Train Loss: 28.7063, Val Loss: 29.3735
Epoch 6/300, Train Loss: 28.7008, Val Loss: 29.3697
Epoch 7/300, Train Loss: 28.6914, Val Loss: 29.3649
Epoch 8/300, Train Loss: 28.7122, Val Loss: 29.3600
Epoch 9/300, Train Loss: 28.7037, Val Loss: 29.3578
Epoch 10/300, Train Loss: 28.6950, Val Loss: 29.3405
Epoch 11/300, Train Loss: 28.6859, Val Loss: 29.3309
Epoch 12/300, Train Loss: 28.6786, Val Loss: 29.3208
Epoch 13/300, Train Loss: 28.7006, Val Loss: 29.3090
Epoch 14/300, Train Loss: 28.6783, Val Loss: 29.2947
Epoch 15/300, Train Loss: 28.6707, Val Loss: 29.2635
Epoch 16/300, Train Loss: 28.6230, Val Loss: 29.2227
Epoch 17/300, Train Loss: 28.5981, Val Loss: 29.1606
Epoch 18/300, Train Loss: 28.5724, Val Loss: 29.0829
Epoch 19/300, Train Loss: 28.5283, Val Loss: 28.9260
Ep

In [15]:
## plot dos pontos de teste e a função f(x) usando o plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_test[:,0], y=x_test[:,1], mode='markers', marker=dict(color=y_test, size=3), name='Dados de teste'))
x = np.linspace(0,4,100)
y = f(x)
fig.add_trace(go.Scatter(x=x, y=y, mode='lines', line=dict(color='orange'), name='f(x)'))
fig.update_layout(title='Dados de teste e f(x)', xaxis_title='X', yaxis_title='Y', height=600, width=900)
fig.show()

In [16]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(train_losses)), y=train_losses, mode='lines', name='Train Loss'))
fig.add_trace(go.Scatter(x=np.arange(len(val_losses)), y=val_losses, mode='lines', name='Val Loss'))
fig.update_layout(title='Train and Val Loss', xaxis_title='Epoch', yaxis_title='Loss', height=600, width=1200)
fig.add_trace(go.Scatter(x=[len(train_losses)], y=[test_loss], mode='markers', marker=dict(color='orange', size=10), name='Test Loss'))
fig.show()